In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import import_ipynb
import WikiCrawler as wk
import DataCleaner as dc
import pandas as pd
from bs4 import BeautifulSoup

importing Jupyter notebook from WikiCrawler.ipynb
importing Jupyter notebook from DataCleaner.ipynb


In [2]:
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
analyzer = SentimentIntensityAnalyzer()
url = "https://en.wikipedia.org/wiki/2019"

SentimentAnalyzerScore returns the sentiment score of the text.

In [4]:
def sentimentAnalyzerScore(text):
    score = analyzer.polarity_scores(text)
    return score

In [52]:
score = analyzer.polarity_scores("hi this is a test text to check how to extraxt compound score")
print(score)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


textMiner extracts the title and context from the Wikipedia page and stores it in a dataframe.

In [5]:
def textMiner(url):
    response = wk.initResp(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = wk.titleExtractor(soup)
    context = wk.contextExtractor(soup)
    wikiPageData = pd.DataFrame([[title,context]])
    dc.lowerOnly(wikiPageData)
    dc.alphaCharOnly(wikiPageData)
    return wikiPageData

stopWordDel removes all the stopwords from a given sentence and returns a filtered sentence.

In [6]:
def stopWordDel(sentence):
    stop_words = set(stopwords.words('english'))
    filteredSent = " "
    pageWord = sentence.split()
    for r in pageWord:
        if not r in stop_words:
            filteredSent = filteredSent + " " + r
    return filteredSent


In [94]:
def posNegWordList(sentence):
    wordList = pd.DataFrame(columns=['word','score'])
    allWords = sentence.split()
    for j in allWords:
        if not j in list(wordList.iloc[:,0]):
            wordList = wordList.append({'word':j,'score':sentimentAnalyzerScore(j)['compound']} , ignore_index=True)
    wordList.sort_values(by=['score'] , inplace=True)
    print("Negative Words: ","\n", wordList.iloc[0:9,0].to_string(index=False))
    print("Positive Words: ","\n",wordList.iloc[-10:-1,0].to_string(index=False))

getPageSentiment combines all three functions and returns the sentiment score of the passed Wikipedia page url.

In [35]:
def getPageSentiment(url):
    df = textMiner(url)
    text = stopWordDel(df.loc[0,1])
    posNegWordList(text)
    score = sentimentAnalyzerScore(text)
    return score

In [97]:
print(" title: ", textMiner("https://en.wikipedia.org/wiki/Black_Death").iloc[0,0]," \n score:",
    getPageSentiment(url = "https://en.wikipedia.org/wiki/Black_Death"))

print("\n\n\n")

print( " title: ", textMiner("https://en.wikipedia.org/wiki/Independence_Day_(India)").iloc[0,0]," \n score:",
      getPageSentiment(url = "https://en.wikipedia.org/wiki/Independence_Day_(India)"))

print("\n\n\n")

print(" title: ", textMiner("https://en.wikipedia.org/wiki/Mona_Lisa").iloc[0,0]," \n score:",
    getPageSentiment(url = "https://en.wikipedia.org/wiki/Mona_Lisa"))


Negative Words:  
    killed
 fatality
 murdered
  killing
     dead
    worst
   crisis
 disaster
 betrayed
Positive Words:  
    survived
       free
    winning
 remarkable
     divine
      great
   paradise
    freedom
        143
 title:    black death wikipedia  
 score: {'neg': 0.158, 'neu': 0.779, 'pos': 0.062, 'compound': -0.9999}




Negative Words:  
  terrorist
  violence
       war
   violent
      died
     fired
     riots
  collapse
  troubled
Positive Words:  
     peaceful
       prize
 encouraging
       peace
  celebrated
 celebrating
      honour
   celebrate
  celebrates
 title:    independence day india wikipedia  
 score: {'neg': 0.084, 'neu': 0.828, 'pos': 0.088, 'compound': -0.1082}




Negative Words:  
        dead
      death
        war
 distraught
        ass
      crime
  painfully
    failure
     prison
Positive Words:  
       honour
  celebrated
       happy
      beauty
   adoration
 masterpiece
       great
         143
        love
 title:    mon